### Iterators

지난 강의에서 우리는 `next`이라는 개념을 사용하여 컬렉션을 반복하는 것에 접근할 수 있다는 것을 보았다.

하지만 아직 해결되지 않은 몇 가지 단점들이 있었습니다!
* 우리는 "for" 루프를 사용할 수 없습니다.
* 우리가 반복을 소진하면(아마도 다음 호출), 우리는 본질적으로 객체에 대해 끝납니다.
개체를 다시 반복하는 유일한 방법은 개체의 새 인스턴스를 만드는 것입니다.

먼저 for 루프에서 next를 사용할 수 있도록 하는 방법을 살펴봅시다.

`_next__`와 `StopIteration` 예외를 사용하는 이 아이디어는 정확히 파이썬이 하는 것입니다.

그래서 어떻게든 우리가 다루고 있는 객체가 `next`와 함께 사용될 수 있다는 것을 파이썬에 알려야 합니다.

이를 위해 우리는 `iterator` 타입의 객체를 만들어야합니다.

Iterators 는 아래를 구현하는 객체입니다
* `__next__` method
* 단순히 객체 자체를 반환하는 `__iter__` method

이것이 다 입니다.
iterator에게 있는 것은 `__iter__` and `__next__` 이라는 두 가지 메소드 그것 뿐입니다.


`Squares` 예제로 돌아갑시다.

In [1]:
class Squares:
    def __init__(self, length):
        self.length = length
        self.i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.length:
            raise StopIteration
        else:
            result = self.i ** 2
            self.i += 1
            return result

우리는 여전히 `next`를 호출할 수 있습니다.

In [2]:
sq = Squares(5)

In [3]:
print(next(sq))
print(next(sq))
print(next(sq))

0
1
4


물론, 우리의 반복자는 여전히 그것을 "restart"할 수 없어 어려움을 겪고 있습니다. 우리는 단지 새로운 인스턴스를 만들어야 합니다:

In [4]:
sq = Squares(5)

그러나 이제는 for 루프를 사용할 수도 있습니다:

In [5]:
for item in sq:
    print(item)

0
1
4
9
16


이제 `sq` 는 **소진되었으므로**, 다시 루프를 돌리면

In [6]:
for item in sq:
    print(item)

우리는 얻을 수 없습니다.

새 iterator를 만들기만 하면 됩니다.

In [7]:
sq = Squares(5)

In [8]:
for item in sq:
    print(item)

0
1
4
9
16


파이썬의 내장 `next` 함수가 우리의 `__next__` 메서드를 호출하는 것처럼 파이썬은 내장 함수 `iter`를 가지고 있으며, 이 함수는 `__iter__` 메서드를 호출합니다:

In [9]:
sq = Squares(5)

In [10]:
id(sq)

1965579635736

In [11]:
id(sq.__iter__())

1965579635736

In [12]:
id(iter(sq))

1965579635736

그리고 물론 우리는 우리의 iterator 객체에 list comprehension을 사용할 수도 있습니다.

In [13]:
sq = Squares(5)

In [14]:
[item for item in sq if item%2==0]

[0, 4, 16]

우리는 심지어 함수에 반복가능한 매개변수를 사용할 수 있습니다.


In [15]:
sq = Squares(5)
list(enumerate(sq))

[(0, 0), (1, 1), (2, 4), (3, 9), (4, 16)]

물론 모두 소진되고 다시 시도할 때는 조심해야합니다.

In [16]:
list(enumerate(sq))

[]

빈 리스트가 표시됩니다. 대신 새 iterator를 먼저 만들어야 합니다.

In [17]:
sq = Squares(5)
list(enumerate(sq))

[(0, 0), (1, 1), (2, 4), (3, 9), (4, 16)]

우리는 심지어 그것에 `sorted` 함수를 사용할 수 있습니다.

In [18]:
sq = Squares(5)
sorted(sq, reverse=True)

[16, 9, 4, 1, 0]

#### Python Iterators Summary

Iterators 는 `__iter__` 과 `__next__` 메소드를 구현하는 객체입니다.

iterator의 `__iter__` 메소드는 단지 자기자신을 리턴합니다.

반복을 통해 모두 **소진**하면 더 이상 반복할 수 없습니다.

파이썬이 반복자 객체에 `for` 루프를 적용하는 방식은 기본적으로 `while` 루프와 `StopIteration` 예외를 통해 본 것입니다.

In [19]:
sq = Squares(5)
while True:
    try:
        print(next(sq))
    except StopIteration:
        break

0
1
4
9
16


실제로 우리는 `iterator`를 조금만 조정하면 이를 쉽게 확인할 수 있습니다

In [20]:
class Squares:
    def __init__(self, length):
        self.length = length
        self.i = 0
        
    def __iter__(self):
        print('calling __iter__')
        return self
    
    def __next__(self):
        print('calling __next__')
        if self.i >= self.length:
            raise StopIteration
        else:
            result = self.i ** 2
            self.i += 1
            return result

In [21]:
sq = Squares(5)

In [22]:
for i in sq:
    print(i)

calling __iter__
calling __next__
0
calling __next__
1
calling __next__
4
calling __next__
9
calling __next__
16
calling __next__


파이썬이 `__next__`를 호출하는 것을 볼 수 있듯이(`StopItation` 예외가 발생하면 중지됨).

마찬가지로 `__iter__` 메소드가 호출되었다는 것을 알 수 있습니다.

사실 우리는 이것이 다른 곳에서도 일어나는 것을 보게 될 것입니다.

In [23]:
sq = Squares(5)
[item for item in sq if item%2==0]

calling __iter__
calling __next__
calling __next__
calling __next__
calling __next__
calling __next__
calling __next__


[0, 4, 16]

In [24]:
sq = Squares(5)
list(enumerate(sq))

calling __iter__
calling __next__
calling __next__
calling __next__
calling __next__
calling __next__
calling __next__


[(0, 0), (1, 1), (2, 4), (3, 9), (4, 16)]

In [25]:
sq = Squares(5)
sorted(sq, reverse=True)

calling __iter__
calling __next__
calling __next__
calling __next__
calling __next__
calling __next__
calling __next__


[16, 9, 4, 1, 0]

왜 `__iter__`이 호출됩니까?
Why is `__iter__` being called? After all, it just returns itself!

그것이 다음 강의의 주제입니다.

그러나 python이 수행하는 작업을 어떻게 모방할 수 있는지 알아보겠습니다.

In [26]:
sq = Squares(5)
sq_iterator = iter(sq)
print(id(sq), id(sq_iterator))
while True:
    try:
        item = next(sq_iterator)
        print(item)
    except StopIteration:
        break

calling __iter__
1965579704808 1965579704808
calling __next__
0
calling __next__
1
calling __next__
4
calling __next__
9
calling __next__
16
calling __next__


보시는 바와 같이 먼저 `iter` 함수를 사용하여 `sq`에서 iterator를 요청합니다. 그리고 나서 iterate는 반환된 iterator를 사용합니다. iterator의 경우 `iter` 함수는 iterator 자체를 다시 가져옵니다.